In [1]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cupyx.scipy as cuxsp
import numpy as np
import cupy as cp
import scipy as sp
from numba import njit
import h5py



In [2]:
cluster = LocalCUDACluster()
client = Client(cluster)
# client = Client("127.0.0.1:8787")

2022-05-10 20:36:16,521 - distributed.diskutils - INFO - Found stale lock file and directory '/home/quojinhao/QOPTICS/Code/python/dask-worker-space/worker-4f6cysp5', purging
2022-05-10 20:36:16,521 - distributed.diskutils - INFO - Found stale lock file and directory '/home/quojinhao/QOPTICS/Code/python/dask-worker-space/worker-rt6csoip', purging
2022-05-10 20:36:16,522 - distributed.diskutils - INFO - Found stale lock file and directory '/home/quojinhao/QOPTICS/Code/python/dask-worker-space/worker-vin3kgr7', purging
2022-05-10 20:36:16,523 - distributed.diskutils - INFO - Found stale lock file and directory '/home/quojinhao/QOPTICS/Code/python/dask-worker-space/worker-8hihwrwj', purging
2022-05-10 20:36:16,524 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-10 20:36:16,525 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-10 20:36:16,528 - distributed.preloading - INFO - Import preload module: dask_cuda.initializ

In [6]:
Nx = 50
Ny = 50
Nz = 50
Lx = 1
Ly = 1
Lz = 1


def vector_GPU(a):
    a = cp.array(a) 
    x = cp.linspace(-Lx,Lx,Nx)
    y = cp.linspace(-Ly,Ly,Ny)
    z = cp.linspace(-Lz,Lz,Nz)
    dx = cp.diff(x)[0]
    dy = cp.diff(y)[0]
    dz = cp.diff(z)[0]
    [X,Y,Z] = cp.meshgrid(x,y,z)
    w =  (X**2 + Y**2 + Z**2)
    w = w[cp.newaxis, ...]
    w = cp.repeat(w,len(a),axis=0)
    w = np.einsum("i,ijkl->ijkl",a,w)
    # boundary is zero
    u=cp.zeros_like(w)
    for i in range(10000):
        u[:, 1:Ny-1,1:Nx-1,1:Nz-1] = (
            (1/dy**2)*(
                    w[:, 2:Ny,   1:Nx-1, 1:Nz-1] 
                - 2*w[:, 1:Ny-1, 1:Nx-1, 1:Nz-1] 
                + w[:, 0:Ny-2, 1:Nx-1, 1:Nz-1])
            +(1/dx**2)*(
                    w[:, 1:Ny-1, 2:Nx,   1:Nz-1] 
                - 2*w[:, 1:Ny-1, 1:Nx-1, 1:Nz-1] 
                + w[:, 1:Ny-1, 0:Nx-2, 1:Nz-1])
            +(1/dz**2)*(
                    w[:, 1:Ny-1, 1:Nx-1, 2:Nz]
                - 2*w[:, 1:Ny-1, 1:Nx-1, 1:Nz-1] 
                + w[:, 1:Ny-1, 1:Nx-1, 0:Nz-2]))
    return u.get()

# futures = []
a = np.array([1e-5,1e-4,1e-3,1e-2,1e-1])
future =  client.submit(vector_GPU,np.repeat(a,30)) 
future1 =  client.submit(vector_GPU,np.repeat(a+1,1)) 
future2 =  client.submit(vector_GPU,np.repeat(a+2,1)) 
future3 =  client.submit(vector_GPU,np.repeat(a+3,1)) 

In [8]:
import computation2_cluster as cc

In [12]:
L1 = np.array([1,2])
L2 = np.array([3,4])
L3 = np.array([5,6])
L4 = np.array([7,8])

future =  client.submit(cc.computation,L1,1000,500,"test1") 
future1 =  client.submit(cc.computation,L2,1000,500,"test2") 
future2 =  client.submit(cc.computation,L3,1000,500,"test3") 
future3 =  client.submit(cc.computation,L4,1000,500,"test4") 


storing succeeded!
storing succeeded!
storing succeeded!
storing succeeded!


In [11]:
client.restart()
# client.close()
# cluster.close()

2022-05-10 20:41:44,504 - distributed.nanny - WARNING - Worker process still alive after 1.5999990463256837 seconds, killing
2022-05-10 20:41:44,505 - distributed.nanny - WARNING - Worker process still alive after 1.5999982833862305 seconds, killing
2022-05-10 20:41:44,506 - distributed.nanny - WARNING - Worker process still alive after 1.5999994277954102 seconds, killing
2022-05-10 20:41:44,506 - distributed.nanny - WARNING - Worker process still alive after 1.5999996185302736 seconds, killing
2022-05-10 20:41:45,572 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-10 20:41:45,586 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-10 20:41:45,621 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-10 20:41:45,626 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 251.30 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40343,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: 5 minutes ago,Total memory: 251.30 GiB
Comm: tcp://127.0.0.1:38473,Total threads: 1
Dashboard: http://127.0.0.1:37467/status,Memory: 62.82 GiB
Nanny: tcp://127.0.0.1:36761,


In [ ]:
with h5py.File(cc.path, "r") as f:
    psiG = f['psiG'][...]
    psiE = f['psiE'][...]   
    LG = f['LG'][...] 
    Nx = f['Metaparameters/Nx'][()]
    Ny = f['Metaparameters/Ny'][()]
    Nz = f['Metaparameters/Nz'][()]
    Lx = f['Metaparameters/Lx'][()]
    Ly = f['Metaparameters/Ly'][()]
    Lz = f['Metaparameters/Lz'][()]
    dw = f['Metaparameters/dw'][()]
    As = f['Parameters/As'][()]
    Nbec = f['Parameters/Nbec'][()]
    Rabi = f['Parameters/Rabi'][()]
    Wx = f['Parameters/Wx'][()]
    Wy = f['Parameters/Wy'][()]
    Wz = f['Parameters/Wz'][()]
    dw = f['Parameters/dw'][()]
    Ggg = f['Parameters/Ggg'][()]
    Gee = f['Parameters/Gee'][()]
    Gge = f['Parameters/Gge'][()]
    Geg = f['Parameters/Geg'][()]
    m = f['Parameters/m'][()]
    print("retrieving succeeded!")
x = np.linspace(-Lx,Lx,Nx)
y = np.linspace(-Ly,Ly,Ny)
z = np.linspace(-Lz,Lz,Nz)
dx = np.diff(x)[0]
dy = np.diff(y)[0]
dz = np.diff(z)[0]
X,Y,Z = np.meshgrid(x,y,z)
# Some constants
pi = 3.14159265359
hbar = 1.054571800139113e-34 
m = 1.411000000000000e-25  # Rb atoms
# BEC parameters
As = 5.82e-09
Nbec = 10000
Rabi = 1000
Wx = 2000
Wy = 2000
Wz = 2000
# unit = 1.222614572474304e-06;
unit = np.sqrt(hbar/m/Wz)

Ggg = (4*pi*hbar**2*As*Nbec/m)*unit**-3*(hbar*Wz)**-1
Gee = Ggg  
Gge = 0
Geg = 0
Epot = ( (Wx**2*X**2 + Wy**2*Y**2 + Wz**2*Z**2 )
            / (2*Wz**2) )
# circular potential
psiGmu = (15*Ggg / ( 16*pi*np.sqrt(2) )  )**(2/5)  
psiEmu = (15*Gee / ( 16*pi*np.sqrt(2) )  )**(2/5)

TF_amp = np.array((psiGmu-Epot)/Ggg,dtype=np.cfloat)
np.clip(TF_amp, 0, np.inf,out=TF_amp)
TF_pbb = np.sqrt(TF_amp)
total = np.sum(np.abs(TF_pbb)**2*dx*dy*dz)
n_TF_pbb = TF_pbb/np.sqrt(total)

print(np.sum(np.abs(TF_pbb)**2*dx*dy*dz))
print(np.sum(np.abs(psiG)**2*dx*dy*dz))
print(np.sum(np.abs(psiE)**2*dx*dy*dz))